In [0]:
from sklearn import datasets
iris = datasets.load_iris()
iris

In [0]:
entrada = iris.data

In [0]:
saida_esperada = iris.target
saida_esperada

In [0]:
from sklearn.preprocessing import StandardScaler

scaler_x = StandardScaler()
entrada = scaler_x.fit_transform(entrada)
entrada

In [0]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categorical_features=[0])

In [0]:
saida_esperada = saida_esperada.reshape(-1, 1)

In [0]:
saida_esperada = onehot.fit_transform(saida_esperada).toarray()

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [0]:
saida_esperada

In [0]:
from sklearn.model_selection import train_test_split
entrada_treinamento, entrada_teste, saida_esperada_treinamento, saida_esperada_teste = train_test_split(entrada, saida_esperada, test_size=0.3)

In [0]:
entrada_treinamento.shape

(105, 4)

In [0]:
entrada_teste.shape

(45, 4)

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
neuronio_entrada = entrada.shape[1] # shape[1] pega a quantidade de colunas da base de dados


In [0]:
neuronios_oculta = int(np.ceil((entrada.shape[1] + float(saida_esperada.shape[1])) / 2))
neuronios_oculta

4

In [0]:
neuronio_saida = saida_esperada.shape[1]
neuronio_saida

3

In [0]:
pesos = {'oculta': tf.Variable(tf.random_normal([neuronio_entrada, neuronios_oculta])),
         'saida': tf.Variable(tf.random_normal([neuronios_oculta, neuronio_saida]))}

In [0]:
bias =  {'oculta': tf.Variable(tf.random_normal([neuronios_oculta])),
         'saida': tf.Variable(tf.random_normal([neuronio_saida]))}

In [0]:
xph = tf.placeholder('float', [None, neuronio_entrada])
yph = tf.placeholder('float', [None, neuronio_saida])

In [0]:
def mpl(x, W, bias):
  camada_oculta = tf.add(tf.matmul(x, W['oculta']), bias['oculta'])
  camada_oculta_ativacao = tf.nn.relu(camada_oculta)
  camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, W['saida']), bias['saida'])
  return camada_saida

In [0]:
modelo = mpl(xph, pesos, bias)

In [0]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(erro)

In [0]:
batch_size = 8
batch_total = int(len(entrada_treinamento) / batch_size )
batch_total

13

In [0]:
x_batches = np.array_split(entrada_treinamento, batch_total)
x_batches

In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for epoca in range(5000):
    erro_medio = 0.0
    batch_total = int(len(entrada_treinamento) / batch_size )
    x_batches = np.array_split(entrada_treinamento, batch_total)
    y_batches = np.array_split(saida_esperada_treinamento, batch_total)
    for i in range(batch_total):
      x_batch, y_batch = x_batches[i], y_batches[i]
      _, custo = sess.run([otimizador, erro],  feed_dict = {xph: x_batch, yph: y_batch})
      erro_medio = custo / batch_total
    if epoca % 500 == 0:
      print("Época: "+str(epoca+1)+" erro: "+str(erro_medio))
  pesos_finais, bias_finais = sess.run([pesos, bias])

Época: 1 erro: 0.2917934564443735
Época: 501 erro: 0.09084453949561486
Época: 1001 erro: 0.05790981421103844
Época: 1501 erro: 0.035849532255759604
Época: 2001 erro: 0.02086613728449895
Época: 2501 erro: 0.01364449354318472
Época: 3001 erro: 0.010445162653923035
Época: 3501 erro: 0.008657934000858894
Época: 4001 erro: 0.007457289558190566
Época: 4501 erro: 0.006623483048035548


In [0]:
pesos_finais

{'oculta': array([[ 0.25009117, -1.6191481 , -0.57565516,  0.19313858],
        [ 0.1625505 ,  0.859963  ,  0.21488094, -1.4591326 ],
        [-1.6351751 , -0.57637495, -1.9704815 , -1.2996233 ],
        [-0.5895837 , -1.3293751 , -0.6387769 , -1.1125704 ]],
       dtype=float32), 'saida': array([[-0.9073538 ,  1.8559643 , -3.458621  ],
        [ 2.5490925 , -1.2346815 , -0.10805469],
        [ 2.2174919 , -1.0323691 , -0.86463255],
        [ 0.31486657,  1.41924   , -2.5081139 ]], dtype=float32)}

In [0]:
bias_finais

{'oculta': array([ 1.8040555 , -0.6061174 , -0.16913892, -1.5588259 ], dtype=float32),
 'saida': array([-3.987417 , -0.7832958,  1.677258 ], dtype=float32)}

In [0]:
#previsões
previsoes_teste = mpl(xph,pesos_finais, bias_finais)
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  r1 = sess.run(previsoes_teste, feed_dict={xph: entrada_teste})
  r2 = sess.run(tf.nn.softmax(r1))#transforma os dados em dados de probabilidade
  r3 = sess.run(tf.argmax(r2, 1))# Transforma em probabilidades em números legíveis, o argmax retorna o indice de maior valor,o "tf.argmax(r2, 1)" quer dizer que você pega o maior valor por coluna

In [0]:
r1

In [0]:
r2

In [0]:
r3

In [0]:
saida_esperada_teste

In [0]:
saida_esperada_teste2 = np.argmax(saida_esperada_teste, 1)
saida_esperada_teste2

array([2, 1, 0, 0, 2, 2, 0, 2, 1, 0, 1, 2, 0, 2, 1, 1, 1, 1, 0, 1, 1, 2,
       1, 2, 0, 0, 0, 2, 1, 0, 0, 2, 0, 1, 1, 0, 2, 0, 2, 2, 1, 1, 2, 0,
       1])

In [0]:
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(saida_esperada_teste2, r3)
taxa_acerto

0.9777777777777777